## SECTION 1: INSTALL AND IMPORT LIBRARIES



In [ ]:
# Install required packages (run this cell first in Colab)
!pip install faker bcrypt ipywidgets sqlalchemy -q

In [ ]:
# Import all necessary libraries
import sqlite3
import pandas as pd
import numpy as np
from faker import Faker
import bcrypt
from datetime import datetime, timedelta
import random
# --- NEW: adding RBAC + login imports
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Engine
from functools import wraps
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
# --- NEW: adding RBAC + login engine
# Autumn Erwin
engine: Engine = create_engine("sqlite:///school_system.db", future=True)

In [ ]:
# Initialize Faker for generating realistic mock data
fake = Faker()
Faker.seed(42)  # Set seed for reproducible data
random.seed(42)

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


 ## SECTION 2: DATABASE SETUP - CREATE ALL TABLES

In [ ]:
# Connect to SQLite database (creates file if doesn't exist)
conn = sqlite3.connect("school_system.db")
cursor = conn.cursor()

In [ ]:
# Drop existing tables for clean slate (useful during development)
tables_to_drop = [
    "ProgressReports", "Alerts", "AfterHoursRequests", "EngagementRequests",
    "Announcements", "Grades", "Assignments", "TeacherAvailability",
    "Courses", "Parent_Student", "Students", "Teachers", "Parents", "Users"
]

In [ ]:
for table in tables_to_drop:
    cursor.execute(f"DROP TABLE IF EXISTS {table}")

print("🗑️  Old tables dropped (if they existed)")

🗑️  Old tables dropped (if they existed)


#### --- CORE TABLES ---

In [ ]:
# Users table: Stores all system users (students, parents, teachers, admins)
cursor.execute("""
CREATE TABLE Users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    password_hash TEXT NOT NULL,
    role TEXT CHECK(role IN ('student','parent','teacher','admin')) NOT NULL
)
""")

In [ ]:
# Students table: Extended info for users who are students
cursor.execute("""
CREATE TABLE Students (
    student_id INTEGER PRIMARY KEY,
    user_id INTEGER NOT NULL,
    grade_level INTEGER,
    enrollment_date TEXT,
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
)
""")

In [ ]:
# Teachers table: Extended info for users who are teachers
cursor.execute("""
CREATE TABLE Teachers (
    teacher_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    department TEXT,
    hire_date TEXT,
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
)
""")

In [ ]:
# Parents table: Extended info for users who are parents
cursor.execute("""
CREATE TABLE Parents (
    parent_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    phone TEXT,
    address TEXT,
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
)
""")

In [ ]:
# Parent_Student: Links parents to their children (many-to-many relationship)
cursor.execute("""
CREATE TABLE Parent_Student (
    parent_id INTEGER,
    student_id INTEGER,
    relationship TEXT,
    PRIMARY KEY (parent_id, student_id),
    FOREIGN KEY (parent_id) REFERENCES Parents(parent_id),
    FOREIGN KEY (student_id) REFERENCES Students(student_id)
)
""")

In [ ]:
# Courses table: All courses offered in the school
cursor.execute("""
CREATE TABLE Courses (
    course_id INTEGER PRIMARY KEY AUTOINCREMENT,
    course_name TEXT NOT NULL,
    course_code TEXT UNIQUE,
    teacher_id INTEGER,
    semester TEXT,
    FOREIGN KEY (teacher_id) REFERENCES Teachers(teacher_id)
)
""")

#### Demo 1 FEATURE TABLES

In [ ]:
# Grades table: Student grades for assignments/exams
cursor.execute("""
CREATE TABLE Grades (
    grade_id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_id INTEGER NOT NULL,
    course_id INTEGER NOT NULL,
    assignment_name TEXT,
    grade_value REAL,
    max_points REAL,
    grade_date TEXT,
    category TEXT,
    FOREIGN KEY (student_id) REFERENCES Students(student_id),
    FOREIGN KEY (course_id) REFERENCES Courses(course_id)
)
""")

In [ ]:
# Announcements table: Teacher/admin announcements (Meetika's feature)
cursor.execute("""
CREATE TABLE Announcements (
    announcement_id INTEGER PRIMARY KEY AUTOINCREMENT,
    author_id INTEGER NOT NULL,
    role_visibility TEXT,
    course_id INTEGER,
    title TEXT NOT NULL,
    body TEXT,
    created_at TEXT,
    FOREIGN KEY (author_id) REFERENCES Users(user_id),
    FOREIGN KEY (course_id) REFERENCES Courses(course_id)
)
""")

In [ ]:
# EngagementRequests table: Parent requests to contact teachers (Keith's feature)
cursor.execute("""
CREATE TABLE EngagementRequests (
    request_id INTEGER PRIMARY KEY AUTOINCREMENT,
    parent_id INTEGER NOT NULL,
    teacher_id INTEGER NOT NULL,
    student_id INTEGER NOT NULL,
    request_type TEXT CHECK(request_type IN ('meeting','message')),
    subject TEXT,
    message TEXT,
    preferred_times TEXT,
    status TEXT CHECK(status IN ('pending','approved','declined','completed')) DEFAULT 'pending',
    created_at TEXT,
    teacher_response TEXT,
    FOREIGN KEY (parent_id) REFERENCES Parents(parent_id),
    FOREIGN KEY (teacher_id) REFERENCES Teachers(teacher_id),
    FOREIGN KEY (student_id) REFERENCES Students(student_id)
)
""")

In [ ]:
# AfterHoursRequests table: After-hours questions (Jaikishan's feature)
cursor.execute("""
CREATE TABLE AfterHoursRequests (
    request_id INTEGER PRIMARY KEY AUTOINCREMENT,
    requester_id INTEGER NOT NULL,
    requester_role TEXT,
    teacher_id INTEGER NOT NULL,
    student_id INTEGER,
    question TEXT,
    submitted_at TEXT,
    status TEXT DEFAULT 'pending',
    teacher_response TEXT,
    response_time TEXT,
    FOREIGN KEY (requester_id) REFERENCES Users(user_id),
    FOREIGN KEY (teacher_id) REFERENCES Teachers(teacher_id)
)
""")

In [ ]:
# TeacherAvailability table: Teacher availability windows
cursor.execute("""
CREATE TABLE TeacherAvailability (
    availability_id INTEGER PRIMARY KEY AUTOINCREMENT,
    teacher_id INTEGER NOT NULL,
    day_of_week TEXT,
    start_time TEXT,
    end_time TEXT,
    FOREIGN KEY (teacher_id) REFERENCES Teachers(teacher_id)
)
""")

#### Demo 2 FEATURE TABLES (FOR FUTURE USE)

In [ ]:
# Assignments table: Upcoming assignments and exams
cursor.execute("""
CREATE TABLE Assignments (
    assignment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    course_id INTEGER NOT NULL,
    title TEXT NOT NULL,
    assignment_type TEXT,
    due_date TEXT,
    weight REAL,
    notes TEXT,
    FOREIGN KEY (course_id) REFERENCES Courses(course_id)
)
""")

In [ ]:
# Alerts table: Low grade alerts
cursor.execute("""
CREATE TABLE Alerts (
    alert_id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_id INTEGER NOT NULL,
    course_id INTEGER,
    alert_type TEXT,
    message TEXT,
    threshold_value REAL,
    created_at TEXT,
    acknowledged INTEGER DEFAULT 0,
    FOREIGN KEY (student_id) REFERENCES Students(student_id)
)
""")

In [ ]:
# ProgressReports table: AI-generated student reports
cursor.execute("""
CREATE TABLE ProgressReports (
    report_id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_id INTEGER NOT NULL,
    generated_at TEXT,
    report_text TEXT,
    metrics_json TEXT,
    FOREIGN KEY (student_id) REFERENCES Students(student_id)
)
""")

In [ ]:
conn.commit()
print("✅ All database tables created successfully!")

✅ All database tables created successfully!


## SECTION 3: GENERATING THE MOCK DATA WITH FAKER

In [ ]:
print("\n📊 Generating realistic mock data...")

# --- STEP 1: Create Users (Students, Parents, Teachers, Admins) ---

# Students: Create 20 students
student_users = []
for i in range(20):
    name = fake.name()
    email = f"student{i+1}@school.edu"
    # Hash password "password123" for all test accounts
    password = bcrypt.hashpw("password123".encode('utf-8'), bcrypt.gensalt()).decode('utf-8')
    cursor.execute("""
        INSERT INTO Users (name, email, password_hash, role)
        VALUES (?, ?, ?, 'student')
    """, (name, email, password))
    student_users.append(cursor.lastrowid)

# Parents: Create 15 parents
parent_users = []
for i in range(15):
    name = fake.name()
    email = f"parent{i+1}@email.com"
    password = bcrypt.hashpw("password123".encode('utf-8'), bcrypt.gensalt()).decode('utf-8')
    cursor.execute("""
        INSERT INTO Users (name, email, password_hash, role)
        VALUES (?, ?, ?, 'parent')
    """, (name, email, password))
    parent_users.append(cursor.lastrowid)

# Teachers: Create 5 teachers
teacher_users = []
departments = ["Mathematics", "English", "Science", "History", "Physical Education"]
for i in range(5):
    name = fake.name()
    email = f"teacher{i+1}@school.edu"
    password = bcrypt.hashpw("password123".encode('utf-8'), bcrypt.gensalt()).decode('utf-8')
    cursor.execute("""
        INSERT INTO Users (name, email, password_hash, role)
        VALUES (?, ?, ?, 'teacher')
    """, (name, email, password))
    teacher_users.append(cursor.lastrowid)

# Admins: Create 2 admins
admin_users = []
for i in range(2):
    name = f"Admin {fake.last_name()}"
    email = f"admin{i+1}@school.edu"
    password = bcrypt.hashpw("password123".encode('utf-8'), bcrypt.gensalt()).decode('utf-8')
    cursor.execute("""
        INSERT INTO Users (name, email, password_hash, role)
        VALUES (?, ?, ?, 'admin')
    """, (name, email, password))
    admin_users.append(cursor.lastrowid)

# --- STEP 2: Create Extended Records for Each Role ---

# Students extended info
student_ids = []
for i, user_id in enumerate(student_users):
    grade_level = random.choice([9, 10, 11, 12])
    enrollment_date = fake.date_between(start_date='-4y', end_date='today').strftime('%Y-%m-%d')
    cursor.execute("""
        INSERT INTO Students (student_id, user_id, grade_level, enrollment_date)
        VALUES (?, ?, ?, ?)
    """, (i+1, user_id, grade_level, enrollment_date))
    student_ids.append(i+1)

# Teachers extended info
teacher_ids = []
for i, user_id in enumerate(teacher_users):
    hire_date = fake.date_between(start_date='-10y', end_date='-1y').strftime('%Y-%m-%d')
    cursor.execute("""
        INSERT INTO Teachers (user_id, department, hire_date)
        VALUES (?, ?, ?)
    """, (user_id, departments[i], hire_date))
    teacher_ids.append(cursor.lastrowid)

# Parents extended info
parent_ids = []
for user_id in parent_users:
    phone = fake.phone_number()
    address = fake.address().replace('\n', ', ')
    cursor.execute("""
        INSERT INTO Parents (user_id, phone, address)
        VALUES (?, ?, ?)
    """, (user_id, phone, address))
    parent_ids.append(cursor.lastrowid)

# --- STEP 3: Link Parents to Students ---

# Each parent has 1-2 children
relationships = ["Mother", "Father", "Guardian"]
for parent_id in parent_ids:
    num_children = random.choice([1, 1, 2])  # Most have 1, some have 2
    children = random.sample(student_ids, num_children)
    for student_id in children:
        relationship = random.choice(relationships)
        cursor.execute("""
            INSERT INTO Parent_Student (parent_id, student_id, relationship)
            VALUES (?, ?, ?)
        """, (parent_id, student_id, relationship))

# --- STEP 4: Create Courses ---

courses_data = [
    ("Algebra II", "MATH201", 1, "Fall 2024"),
    ("English Literature", "ENG101", 2, "Fall 2024"),
    ("Biology", "SCI301", 3, "Fall 2024"),
    ("World History", "HIST201", 4, "Fall 2024"),
    ("Physical Education", "PE101", 5, "Fall 2024"),
    ("Geometry", "MATH101", 1, "Spring 2025"),
]

course_ids = []
for course_name, code, teacher_id, semester in courses_data:
    cursor.execute("""
        INSERT INTO Courses (course_name, course_code, teacher_id, semester)
        VALUES (?, ?, ?, ?)
    """, (course_name, code, teacher_id, semester))
    course_ids.append(cursor.lastrowid)

# --- STEP 5: Generate Grades ---

# Each student gets grades for 3-4 courses
assignment_types = ["Quiz", "Test", "Homework", "Project", "Midterm", "Final"]
categories = ["Homework", "Quizzes", "Tests", "Projects"]

for student_id in student_ids:
    # Enroll student in 3-4 random courses
    student_courses = random.sample(course_ids, random.randint(3, 4))

    for course_id in student_courses:
        # Generate 5-8 grades per course
        num_grades = random.randint(5, 8)
        for _ in range(num_grades):
            assignment_name = f"{random.choice(assignment_types)} {random.randint(1, 10)}"
            max_points = random.choice([10, 20, 50, 100])
            # Generate realistic grade distribution (most students do okay)
            grade_percent = random.gauss(0.80, 0.12)  # Mean 80%, std dev 12%
            grade_percent = max(0.4, min(1.0, grade_percent))  # Clamp between 40-100%
            grade_value = round(grade_percent * max_points, 1)
            grade_date = fake.date_between(start_date='-60d', end_date='today').strftime('%Y-%m-%d')
            category = random.choice(categories)

            cursor.execute("""
                INSERT INTO Grades (student_id, course_id, assignment_name, grade_value, max_points, grade_date, category)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (student_id, course_id, assignment_name, grade_value, max_points, grade_date, category))

# --- STEP 6: Create Sample Announcements ---

announcements_data = [
    (teacher_users[0], "all", None, "Welcome Back!", "Welcome to the new semester! Looking forward to a great year.", "2024-08-15"),
    (teacher_users[1], "student", course_ids[1], "Essay Due Next Week", "Reminder: Your literary analysis essay is due next Monday.", "2024-09-20"),
    (admin_users[0], "all", None, "Parent-Teacher Conferences", "Parent-teacher conferences will be held on October 15th. Sign up through the portal.", "2024-10-01"),
    (teacher_users[2], "student,parent", course_ids[2], "Lab Safety Reminder", "Please review the lab safety guidelines before our next session.", "2024-09-25"),
]

for author_id, visibility, course_id, title, body, created_at in announcements_data:
    cursor.execute("""
        INSERT INTO Announcements (author_id, role_visibility, course_id, title, body, created_at)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (author_id, visibility, course_id, title, body, created_at))

# --- STEP 7: Create Sample Engagement Requests ---

# Create 3 sample engagement requests (some pending, some completed)
sample_requests = [
    (parent_ids[0], teacher_ids[0], student_ids[0], "meeting", "Discussing Math Progress",
     "I'd like to discuss my child's recent quiz scores in Algebra.",
     "Monday 3-4pm, Tuesday 2-3pm", "pending", "2024-10-01", None),
    (parent_ids[1], teacher_ids[1], student_ids[2], "message", "Question about Assignment",
     "Could you clarify the requirements for the upcoming essay?",
     None, "completed", "2024-09-28", "The essay should be 5 pages, MLA format. See rubric for details."),
    (parent_ids[2], teacher_ids[2], student_ids[5], "meeting", "Concerns about Grade",
     "I noticed a drop in my child's Biology grades. Can we meet to discuss?",
     "Wednesday after 3pm, Thursday after 3pm", "approved", "2024-10-02", "Meeting scheduled for Wednesday at 3:30pm."),
]

for req in sample_requests:
    cursor.execute("""
        INSERT INTO EngagementRequests
        (parent_id, teacher_id, student_id, request_type, subject, message, preferred_times, status, created_at, teacher_response)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, req)

# --- STEP 8: Create Teacher Availability ---

# Set availability for all teachers (Monday-Friday, 3-5pm)
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
for teacher_id in teacher_ids:
    for day in days:
        cursor.execute("""
            INSERT INTO TeacherAvailability (teacher_id, day_of_week, start_time, end_time)
            VALUES (?, ?, ?, ?)
        """, (teacher_id, day, "15:00", "17:00"))

conn.commit()
print("✅ Mock data generated successfully!")
print(f"   - {len(student_users)} students")
print(f"   - {len(parent_users)} parents")
print(f"   - {len(teacher_users)} teachers")
print(f"   - {len(admin_users)} admins")
print(f"   - {len(course_ids)} courses")
print(f"   - ~{len(student_ids) * 20} grade entries")


📊 Generating realistic mock data...
✅ Mock data generated successfully!
   - 20 students
   - 15 parents
   - 5 teachers
   - 2 admins
   - 6 courses
   - ~400 grade entries


## SECTION 4: AUTHENTICATION SYSTEM

In [ ]:
# --- NEW: adding RBAC + login helpers
# Autumn Erwin
def require_login(fn):
    """Decorator to ensure a user is logged in before running a view/action."""
    @wraps(fn)
    def wrapper(*args, **kwargs):
        global current_session
        if not current_session:
            print("❌ You must be logged in to access this feature.")
            return
        return fn(*args, **kwargs)
    return wrapper

def require_role(*allowed_roles):
    """
    Decorator to ensure the logged-in session role is in allowed_roles.
    Usage: @require_role('admin', 'teacher')
    """
    def deco(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            global current_session
            if not current_session or current_session.get('role') not in allowed_roles:
                print(f"❌ Access denied. Allowed role(s): {', '.join(allowed_roles)}.")
                return
            return fn(*args, **kwargs)
        return wrapper
    return deco

@require_login
def whoami():
    """Small helper to show current session info."""
    s = current_session
    print("👤 Logged in as:")
    for k in sorted(s.keys()):
        print(f" - {k}: {s[k]}")

@require_login
def get_authorized_grades_df():
    """
    CENTRALIZED, ROLE-AWARE grades accessor.
    Returns a DataFrame containing only the rows the current user is allowed to see.

    Role behavior:
      - student: only that student's grades
      - parent: all linked child(ren) grades via Parent_Student
      - teacher: grades for courses they teach
      - admin: all grades
    """
    role = current_session['role']

    if role == 'student':
        # Student sees only their own grades
        student_id = current_session['student_id']
        q = """
            SELECT g.*, c.course_name, c.course_code
            FROM Grades g
            JOIN Courses c ON c.course_id = g.course_id
            WHERE g.student_id = ?
            ORDER BY g.grade_date DESC
        """
        return pd.read_sql_query(q, conn, params=(student_id,))

    elif role == 'parent':
        # Parent sees all linked children via Parent_Student
        parent_id = current_session['parent_id']
        # In case current_session already has student_ids, we trust it; else fetch
        student_ids = current_session.get('student_ids', None)
        if not student_ids:
            q_ids = """
                SELECT ps.student_id
                FROM Parent_Student ps
                WHERE ps.parent_id = ?
            """
            df_ids = pd.read_sql_query(q_ids, conn, params=(parent_id,))
            student_ids = df_ids['student_id'].tolist()
            current_session['student_ids'] = student_ids

        if not student_ids:
            return pd.DataFrame(columns=[
                'grade_id','student_id','course_id','assignment_name','grade_value',
                'max_points','grade_date','category','course_name','course_code'
            ])

        placeholders = ",".join("?"*len(student_ids))
        q = f"""
            SELECT g.*, c.course_name, c.course_code, u.name AS student_name
            FROM Grades g
            JOIN Courses c ON c.course_id = g.course_id
            JOIN Students s ON s.student_id = g.student_id
            JOIN Users u ON u.user_id = s.user_id
            WHERE g.student_id IN ({placeholders})
            ORDER BY g.grade_date DESC
        """
        return pd.read_sql_query(q, conn, params=tuple(student_ids))

    elif role == 'teacher':
        # Teacher sees grades for their own courses
        teacher_id = current_session['teacher_id']
        q = """
            SELECT g.*, c.course_name, c.course_code, u.name AS student_name
            FROM Grades g
            JOIN Courses c ON c.course_id = g.course_id
            JOIN Teachers t ON t.teacher_id = c.teacher_id
            JOIN Students s ON s.student_id = g.student_id
            JOIN Users u ON u.user_id = s.user_id
            WHERE t.teacher_id = ?
            ORDER BY g.grade_date DESC
        """
        return pd.read_sql_query(q, conn, params=(teacher_id,))

    elif role == 'admin':
        # Admin sees everything
        q = """
            SELECT g.*, c.course_name, c.course_code, u.name AS student_name
            FROM Grades g
            JOIN Courses c ON c.course_id = g.course_id
            JOIN Students s ON s.student_id = g.student_id
            JOIN Users u ON u.user_id = s.user_id
            ORDER BY g.grade_date DESC
        """
        return pd.read_sql_query(q, conn)

    else:
        raise ValueError(f"Unknown role: {role}")

@require_login
def secure_view_grades(limit=20):
    """
    Simple, secure, role-filtered grade viewer that uses the central accessor.
    """
    df = get_authorized_grades_df()
    if df.empty:
        print("📭 No grades available for your permissions.")
        return
    print(f"🔒 Role: {current_session['role']} • Showing {min(limit, len(df))} of {len(df)} rows")
    display(df.head(limit))

In [ ]:
# Global variable to store current session (who is logged in)
current_session = None

def authenticate_user(email, password):
    """
    Authenticate user by checking email and password.
    Returns user info if successful, None if failed.
    """
    # Query database for user with given email
    conn = sqlite3.connect('school_system.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Users WHERE email = ?", (email,))
    user = cursor.fetchone()
    conn.close()

    if user is None:
        return None  # Email not found

    # Extract user info
    user_id, name, stored_email, password_hash, role = user

    # Verify password using bcrypt
    if bcrypt.checkpw(password.encode('utf-8'), password_hash.encode('utf-8')):
        # Password correct! Prepare session data
        session = {
            'user_id': user_id,
            'name': name,
            'email': stored_email,
            'role': role
        }

        conn = sqlite3.connect('school_system.db')
        cursor = conn.cursor()
        # If parent, get their children's IDs
        if role == 'parent':
            cursor.execute("""
                SELECT parent_id FROM Parents WHERE user_id = ?
            """, (user_id,))
            parent_id = cursor.fetchone()[0]

            cursor.execute("""
                SELECT student_id FROM Parent_Student WHERE parent_id = ?
            """, (parent_id,))
            student_ids = [row[0] for row in cursor.fetchall()]
            session['parent_id'] = parent_id
            session['student_ids'] = student_ids

        # If teacher, get their teacher_id
        elif role == 'teacher':
            cursor.execute("""
                SELECT teacher_id FROM Teachers WHERE user_id = ?
            """, (user_id,))
            teacher_id = cursor.fetchone()[0]
            session['teacher_id'] = teacher_id

        # If student, get their student_id
        elif role == 'student':
            cursor.execute("""
                SELECT student_id FROM Students WHERE user_id = ?
            """, (user_id,))
            student_id = cursor.fetchone()[0]
            session['student_id'] = student_id
        conn.close()

        return session
    else:
        return None  # Password incorrect

def logout():
    """Clear current session (log out user)"""
    global current_session
    current_session = None
    print("✅ Logged out successfully!")

# SECTION 5: ANNOUNCEMENTS PAGE

In [ ]:
# ==============================================================
# 📢 ANNOUNCEMENTS PAGE — Meetika Kanumukula
# ==============================================================
# Service: A stream of teacher/admin announcements visible to appropriate audiences.
# Teachers/Admins can post announcements; Students/Parents can only view relevant ones.

import sqlite3
import pandas as pd
from datetime import datetime

# ---- Connect to existing shared database ----
conn = sqlite3.connect('school_system.db')
cursor = conn.cursor()

# ---- Create Announcements table (if it doesn't exist) ----
cursor.execute('''
CREATE TABLE IF NOT EXISTS Announcements (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    author_id INTEGER,
    role_visibility TEXT,
    course_id INTEGER,
    title TEXT,
    body TEXT,
    created_at TEXT
)
''')
conn.commit()


# ==============================================================
#  FUNCTIONS
# ==============================================================

def add_announcement(author_id, author_role):
    """Allow Teachers/Admins to add announcements visible to selected roles."""
    print("\n--- 📝 Add New Announcement ---")
    title = input("Enter announcement title: ").strip()
    body = input("Enter announcement body: ").strip()
    role_visibility = input("Visible to (Student/Parent/Teacher/Admin/All): ").strip().title()
    course_id = input("Enter Course ID (optional, press Enter to skip): ").strip()
    course_id = int(course_id) if course_id else None

    created_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    cursor.execute('''
        INSERT INTO Announcements (author_id, role_visibility, course_id, title, body, created_at)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (author_id, role_visibility, course_id, title, body, created_at))
    conn.commit()

    print("\n✅ Announcement successfully added!\n")


def view_announcements(user_role, course_id=None):
    """Display announcements filtered by the viewer's role (and course if applicable)."""
    print("\n--- 📢 Announcements ---")

    if course_id:
        query = f"""
            SELECT title, body, role_visibility, created_at
            FROM Announcements
            WHERE (role_visibility = '{user_role}' OR role_visibility = 'All')
            AND (course_id IS NULL OR course_id = {course_id})
            ORDER BY datetime(created_at) DESC
        """
    else:
        query = f"""
            SELECT title, body, role_visibility, created_at
            FROM Announcements
            WHERE role_visibility = '{user_role}' OR role_visibility = 'All'
            ORDER BY datetime(created_at) DESC
        """

    df = pd.read_sql_query(query, conn)

    if df.empty:
        print("No announcements available for your role at this time.\n")
    else:
        display(df)


def announcements_menu(current_user_id, current_user_role):
    """Main menu for Announcements section (Teachers/Admins can add; others view)."""
    print("\n📢 Announcements Menu:")
    print("1. View Announcements")
    if current_user_role in ["Teacher", "Admin"]:
        print("2. Add Announcement")

    choice = input("Enter your choice: ").strip()

    if choice == "1":
        view_announcements(current_user_role)
    elif choice == "2" and current_user_role in ["Teacher", "Admin"]:
        add_announcement(current_user_id, current_user_role)
    else:
        print("Invalid option or insufficient permissions.\n")


# ==============================================================
#  DASHBOARD INTEGRATION EXAMPLE
# ==============================================================

def teacher_admin_dashboard(current_user_id, current_user_role):
    """Example integration of Announcements Page into an existing dashboard loop."""
    while True:
        print("\n==============================")
        print(f"Welcome, {current_user_role}!")
        print("==============================")
        print("1. View Students")
        print("2. Update Grades")
        print("3. Send Message")
        print("4. View Messages")
        print("5. 📢 Announcements")
        print("6. Logout")

        choice = input("Enter your choice: ").strip()

        if choice == "1":
            print("🔍 View Students (placeholder for existing function)")
            # view_students()  <-- already defined elsewhere in your system
        elif choice == "2":
            print("📝 Update Grades (placeholder for existing function)")
            # update_grades()
        elif choice == "3":
            print("✉️ Send Message (placeholder for existing function)")
            # send_message()
        elif choice == "4":
            print("📬 View Messages (placeholder for existing function)")
            # view_messages()
        elif choice == "5":
            announcements_menu(current_user_id, current_user_role)
        elif choice == "6":
            print("Logging out...\n")
            break
        else:
            print("❌ Invalid choice. Try again.")


# ==============================================================
#  DEMO SIMULATION (optional)
# ==============================================================

# Uncomment this block to test announcements independently in Colab:
"""
# Simulate a logged-in Teacher
current_user_id = 1
current_user_role = "Teacher"

teacher_admin_dashboard(current_user_id, current_user_role)
"""


'\n# Simulate a logged-in Teacher\ncurrent_user_id = 1\ncurrent_user_role = "Teacher"\n\nteacher_admin_dashboard(current_user_id, current_user_role)\n'

# SECTION 6: LOGIN WIDGET/USER INTERFACE

In [ ]:
# --- NEW: adding RBAC + login panel
# Autumn Erwin
login_email = widgets.Text(
    placeholder="you@example.com", description="Email:", layout=widgets.Layout(width="350px")
)
login_password = widgets.Password(
    placeholder="••••••••", description="Password:", layout=widgets.Layout(width="350px")
)
login_button = widgets.Button(description="Log In", button_style="primary")
login_out = widgets.Output()

def _do_widget_login(_):
    with login_out:
        clear_output()
        email = login_email.value.strip()
        password = login_password.value
        if not email or not password:
            print("⚠️ Please enter both email and password.")
            return

        session = authenticate_user(email, password)  # uses your existing function
        if session:
            global current_session
            current_session = session
            print("✅ Login successful!")
            print(f"👤 {session['name']}  •  Role: {session['role']}")
            # Optional: immediately show a secure view
            print("\n🔎 Sample Role-Filtered Grades:")
            secure_view_grades(limit=10)
        else:
            print("❌ Invalid credentials. Try again. (Hint: password is 'password123' for mock users)")

login_button.on_click(_do_widget_login)

def show_login_widget():
    """Display the GUI login form."""
    display(widgets.VBox([login_email, login_password, login_button, login_out]))

In [ ]:
def login():
    """
    TEXT-BASED login function - no buggy widgets
    Simple and reliable for Google Colab
    """
    global current_session

    print("=" * 60)
    print("🔐 SCHOOL MANAGEMENT SYSTEM - LOGIN")
    print("=" * 60)
    print("\n📝 Test Accounts (all use password: password123):")
    print("   Student: student1@school.edu")
    print("   Parent:  parent1@email.com")
    print("   Teacher: teacher1@school.edu")
    print("   Admin:   admin1@school.edu")
    print("\n")

    email = input("Email: ").strip()
    password = input("Password: ").strip()

    if not email or not password:
        print("❌ Please enter both email and password")
        return

    # Attempt authentication
    session = authenticate_user(email, password)

    if session:
        current_session = session
        print(f"\n✅ Welcome, {session['name']}!")
        print(f"   Role: {session['role'].upper()}")
        print("\n" + "=" * 60)

        # Show appropriate dashboard
        if session['role'] == 'student':
            show_student_dashboard()
        elif session['role'] == 'parent':
            show_parent_dashboard()
        elif session['role'] == 'teacher':
            show_teacher_dashboard()
        elif session['role'] == 'admin':
            show_admin_dashboard()
    else:
        print("\n❌ Invalid email or password. Please try again.")
        print("💡 Hint: All test accounts use password 'password123'")

# SECTION 7: DASHBOARD FUNCTIONS (Based on Actor/Role)

In [ ]:
def show_student_dashboard():
    """Display student dashboard - shows only their own grades"""
    print("\n" + "=" * 60)
    print(f"📚 STUDENT DASHBOARD - {current_session['name']}")
    print("=" * 60)

    student_id = current_session['student_id']

    # Get student's grades
    query = """
        SELECT c.course_name, g.assignment_name, g.grade_value, g.max_points,
               g.grade_date, g.category
        FROM Grades g
        JOIN Courses c ON g.course_id = c.course_id
        WHERE g.student_id = ?
        ORDER BY g.grade_date DESC
        LIMIT 10
    """
    df = pd.read_sql_query(query, conn, params=(student_id,))

    if len(df) > 0:
        df['Percentage'] = ((df['grade_value'] / df['max_points']) * 100).round(1)
        print("\n📊 Recent Grades:")
        print(df[['course_name', 'assignment_name', 'grade_value', 'max_points', 'Percentage', 'grade_date']].to_string(index=False))

        # Calculate overall average
        avg = df['Percentage'].mean()
        print(f"\n📈 Overall Average: {avg:.1f}%")
    else:
        print("\n📭 No grades found.")

def show_teacher_dashboard():
    """Display teacher dashboard - shows their courses and incoming requests"""
    print("\n" + "=" * 60)
    print(f"👨‍🏫 TEACHER DASHBOARD - {current_session['name']}")
    print("=" * 60)

    teacher_id = current_session['teacher_id']

    # Show courses taught
    query = """
        SELECT course_id, course_name, course_code, semester
        FROM Courses
        WHERE teacher_id = ?
    """
    df = pd.read_sql_query(query, conn, params=(teacher_id,))
    print("\n📚 Your Courses:")
    print(df.to_string(index=False))

    # Show incoming engagement requests
    query = """
        SELECT er.request_id, u.name as parent_name, s.student_id, us.name as student_name,
               er.request_type, er.subject, er.status, er.created_at
        FROM EngagementRequests er
        JOIN Parents p ON er.parent_id = p.parent_id
        JOIN Users u ON p.user_id = u.user_id
        JOIN Students s ON er.student_id = s.student_id
        JOIN Users us ON s.user_id = us.user_id
        WHERE er.teacher_id = ?
        ORDER BY er.created_at DESC
    """
    df = pd.read_sql_query(query, conn, params=(teacher_id,))

    if len(df) > 0:
        print("\n📬 Parent Engagement Requests:")
        print(df.to_string(index=False))
    else:
        print("\n📭 No pending requests.")

def show_admin_dashboard():
    """Display admin dashboard - shows system overview"""
    print("\n" + "=" * 60)
    print(f"👔 ADMIN DASHBOARD - {current_session['name']}")
    print("=" * 60)

    # System statistics
    cursor.execute("SELECT COUNT(*) FROM Users WHERE role='student'")
    student_count = cursor.fetchone()[0]

    cursor.execute("SELECT COUNT(*) FROM Users WHERE role='parent'")
    parent_count = cursor.fetchone()[0]

    cursor.execute("SELECT COUNT(*) FROM Users WHERE role='teacher'")
    teacher_count = cursor.fetchone()[0]

    cursor.execute("SELECT COUNT(*) FROM Courses")
    course_count = cursor.fetchone()[0]

    cursor.execute("SELECT COUNT(*) FROM EngagementRequests WHERE status='pending'")
    pending_requests = cursor.fetchone()[0]

    print("\n📊 System Overview:")
    print(f"   Students: {student_count}")
    print(f"   Parents: {parent_count}")
    print(f"   Teachers: {teacher_count}")
    print(f"   Courses: {course_count}")
    print(f"   Pending Engagement Requests: {pending_requests}")

    # Recent activity
    query = """
        SELECT created_at, title, body
        FROM Announcements
        ORDER BY created_at DESC
        LIMIT 5
    """
    df = pd.read_sql_query(query, conn)
    print("\n📢 Recent Announcements:")
    print(df.to_string(index=False))

def show_parent_dashboard():
    """
    Display parent dashboard - KEITH'S MAIN FEATURE
    Shows children's grades and provides engagement tools
    """
    print("\n" + "=" * 60)
    print(f"👨‍👩‍👧 PARENT DASHBOARD - {current_session['name']}")
    print("=" * 60)

    parent_id = current_session['parent_id']
    student_ids = current_session['student_ids']

    # Get children's information
    query = """
        SELECT s.student_id, u.name, s.grade_level
        FROM Students s
        JOIN Users u ON s.user_id = u.user_id
        WHERE s.student_id IN ({})
    """.format(','.join('?' * len(student_ids)))

    children_df = pd.read_sql_query(query, conn, params=student_ids)

    print("\n👶 Your Children:")
    print(children_df.to_string(index=False))

    # Show grades for each child
    for _, child in children_df.iterrows():
        student_id = child['student_id']
        student_name = child['name']

        print(f"\n📊 Grades for {student_name}:")

        query = """
            SELECT c.course_name, g.assignment_name, g.grade_value, g.max_points,
                   g.grade_date, c.teacher_id
            FROM Grades g
            JOIN Courses c ON g.course_id = c.course_id
            WHERE g.student_id = ?
            ORDER BY g.grade_date DESC
            LIMIT 5
        """
        grades_df = pd.read_sql_query(query, conn, params=(student_id,))

        if len(grades_df) > 0:
            grades_df['Percentage'] = ((grades_df['grade_value'] / grades_df['max_points']) * 100).round(1)
            print(grades_df[['course_name', 'assignment_name', 'grade_value', 'max_points', 'Percentage', 'grade_date']].to_string(index=False))

            # Calculate average
            avg = grades_df['Percentage'].mean()
            print(f"   📈 Average: {avg:.1f}%")
        else:
            print("   No grades found.")

    # Show existing engagement requests
    print("\n📬 Your Engagement Request History:")
    query = """
        SELECT er.request_id, t.department, ut.name as teacher_name, us.name as student_name,
               er.request_type, er.subject, er.status, er.created_at
        FROM EngagementRequests er
        JOIN Teachers t ON er.teacher_id = t.teacher_id
        JOIN Users ut ON t.user_id = ut.user_id
        JOIN Students s ON er.student_id = s.student_id
        JOIN Users us ON s.user_id = us.user_id
        WHERE er.parent_id = ?
        ORDER BY er.created_at DESC
    """
    requests_df = pd.read_sql_query(query, conn, params=(parent_id,))

    if len(requests_df) > 0:
        print(requests_df.to_string(index=False))
    else:
        print("   No previous requests.")

    print("\n" + "=" * 60)
    print("🎯 PARENT ENGAGEMENT TOOLS")
    print("=" * 60)
    print("\n💡 Use the function calls below to interact with teachers:")
    print("   parent_contact_teacher()  - Send a message to a teacher")
    print("   parent_request_meeting()  - Request a meeting with a teacher")
    print("   parent_view_requests()    - View all your request statuses")

In [ ]:
# --- NEW: adding RBAC + login secure dashboards that rely on central RBAC accessor
# Autumn Erwin
@require_role('student')
def show_student_dashboard():
    clear_output(wait=True)
    print("="*60)
    print(f"📚 STUDENT DASHBOARD – {current_session['name']}")
    print("="*60)
    df = get_authorized_grades_df()
    if df.empty:
        print("No grades yet.")
        return
    # Show only relevant student columns cleanly
    display(df[['course_name','assignment_name','grade_value','max_points','grade_date','category']].head(25))

@require_role('parent')
def show_parent_dashboard():
    clear_output(wait=True)
    print("="*60)
    print(f"👪 PARENT DASHBOARD – {current_session['name']}")
    print("="*60)
    df = get_authorized_grades_df()
    if df.empty:
        print("No grades for your linked student(s).")
        return
    # Show student_name when available (we add in parent/teacher/admin branches)
    cols = [c for c in ['student_name','course_name','assignment_name','grade_value','max_points','grade_date','category'] if c in df.columns]
    display(df[cols].head(40))

@require_role('teacher')
def show_teacher_dashboard():
    clear_output(wait=True)
    print("="*60)
    print(f"🍎 TEACHER DASHBOARD – {current_session['name']}")
    print("="*60)
    df = get_authorized_grades_df()
    if df.empty:
        print("No grades for your courses.")
        return
    cols = [c for c in ['student_name','course_name','assignment_name','grade_value','max_points','grade_date','category'] if c in df.columns]
    display(df[cols].head(50))

@require_role('admin')
def show_admin_dashboard():
    clear_output(wait=True)
    print("="*60)
    print(f"🏫 ADMIN DASHBOARD – {current_session['name']}")
    print("="*60)
    df = get_authorized_grades_df()
    if df.empty:
        print("No grades in the system.")
        return
    cols = [c for c in ['student_name','course_name','assignment_name','grade_value','max_points','grade_date','category'] if c in df.columns]
    display(df[cols].head(100))

In [ ]:
# --- NEW: adding RBAC + login router to check roles + call correct secure dashboard
# Autumn Erwin
@require_login
def show_my_dashboard():
    """Route to the correct secure dashboard based on the current role."""
    role = current_session['role']
    if role == 'student':
        show_student_dashboard()
    elif role == 'parent':
        show_parent_dashboard()
    elif role == 'teacher':
        show_teacher_dashboard()
    elif role == 'admin':
        show_admin_dashboard()
    else:
        print(f"Unknown role: {role}")

In [ ]:
# --- NEW: adding RBAC + login secure SQL passthrough for future cells (use wrapper to keep RBAC safe)
# Autumn Erwin
def rbac_read_sql(entity: str, base_sql: str, params: tuple = ()):
    """
    Tiny helper to attach WHERE clauses safely based on entity+role, when feasible.
    For now we explicitly support 'grades'. In other cases, return base query.
    """
    role = current_session.get('role') if current_session else None
    if entity != 'grades' or not role:
        # Fallback: no modification
        return pd.read_sql_query(base_sql, conn, params=params)

    if role == 'admin':
        return pd.read_sql_query(base_sql, conn, params=params)

    if role == 'student':
        student_id = current_session['student_id']
        sql = f"SELECT * FROM ({base_sql}) AS t WHERE t.student_id = ?"
        return pd.read_sql_query(sql, conn, params=params + (student_id,))

    if role == 'parent':
        student_ids = current_session.get('student_ids') or []
        if not student_ids:
            # fetch links if missing
            q_ids = "SELECT student_id FROM Parent_Student WHERE parent_id = ?"
            df_ids = pd.read_sql_query(q_ids, conn, params=(current_session['parent_id'],))
            student_ids = df_ids['student_id'].tolist()
            current_session['student_ids'] = student_ids
        if not student_ids:
            return pd.DataFrame()
        placeholders = ",".join("?"*len(student_ids))
        sql = f"SELECT * FROM ({base_sql}) AS t WHERE t.student_id IN ({placeholders})"
        return pd.read_sql_query(sql, conn, params=params + tuple(student_ids))

    if role == 'teacher':
        teacher_id = current_session['teacher_id']
        # assume base_sql has course_id or can be joined externally; we scope by teacher_id via Courses
        sql = f"""
            SELECT t.*
            FROM ({base_sql}) AS t
            JOIN Courses c ON c.course_id = t.course_id
            WHERE c.teacher_id = ?
        """
        return pd.read_sql_query(sql, conn, params=params + (teacher_id,))


# SECTION 8: Parent Engagement Tools (Text Based because widgets was glitchy)

In [ ]:
def parent_contact_teacher():
    """
    TEXT-BASED: Send a message to a teacher
    No buggy widgets - just simple input() prompts
    """
    # Check if logged in as parent
    if not current_session or current_session['role'] != 'parent':
        print("❌ This feature is only available to parents. Please login as a parent first.")
        return

    parent_id = current_session['parent_id']
    student_ids = current_session['student_ids']

    print("\n" + "=" * 60)
    print("📧 CONTACT TEACHER")
    print("=" * 60)

    # Step 1: Select which child
    query = """
        SELECT s.student_id, u.name
        FROM Students s
        JOIN Users u ON s.user_id = u.user_id
        WHERE s.student_id IN ({})
    """.format(','.join('?' * len(student_ids)))
    children_df = pd.read_sql_query(query, conn, params=student_ids)

    print("\n👶 Your Children:")
    children_list = []
    for idx, row in children_df.iterrows():
        children_list.append({'student_id': row['student_id'], 'name': row['name']})
        print(f"   {len(children_list)}. {row['name']} (ID: {row['student_id']})")

    child_choice = input("\nSelect child number: ").strip()
    try:
        choice_num = int(child_choice)
        if choice_num < 1 or choice_num > len(children_list):
            print("❌ Invalid selection")
            return
        student_id = children_list[choice_num - 1]['student_id']
        student_name = children_list[choice_num - 1]['name']
    except:
        print("❌ Invalid input")
        return

    # Step 2: Get teachers for this student
    query = """
        SELECT DISTINCT t.teacher_id, u.name, c.course_name, t.department
        FROM Grades g
        JOIN Courses c ON g.course_id = c.course_id
        JOIN Teachers t ON c.teacher_id = t.teacher_id
        JOIN Users u ON t.user_id = u.user_id
        WHERE g.student_id = ?
    """
    teachers_df = pd.read_sql_query(query, conn, params=(student_id,))

    if len(teachers_df) == 0:
        print(f"❌ No teachers found for {student_name}")
        return

    print(f"\n📚 Teachers for {student_name}:")
    teachers_list = []
    for idx, row in teachers_df.iterrows():
        teachers_list.append({'teacher_id': row['teacher_id'], 'name': row['name']})
        print(f"   {len(teachers_list)}. {row['name']} - {row['course_name']} ({row['department']})")

    teacher_choice = input("\nSelect teacher number: ").strip()
    try:
        choice_num = int(teacher_choice)
        if choice_num < 1 or choice_num > len(teachers_list):
            print("❌ Invalid selection")
            return
        teacher_id = teachers_list[choice_num - 1]['teacher_id']
        teacher_name = teachers_list[choice_num - 1]['name']
    except:
        print("❌ Invalid input")
        return

    # Step 3: Get meeting details
    print(f"\n📝 Meeting request for {teacher_name}...")
    subject = input("What would you like to discuss? ").strip()
    if not subject:
        print("❌ Subject cannot be empty")
        return

    print("\nAdditional details (optional, press Enter to skip): ")
    message = input().strip()

    # Step 4: Get preferred times
    print("\n⏰ Preferred Meeting Times:")
    print("Enter times in format 'Day HH:MM-HH:MM' (e.g., 'Monday 15:00-16:00')")
    print("Enter one time per line. Press Enter on empty line when done.")

    preferred_times = []
    while True:
        time_slot = input(f"Time slot #{len(preferred_times) + 1} (or press Enter to finish): ").strip()
        if time_slot == "":
            break
        preferred_times.append(time_slot)

    if not preferred_times:
        print("❌ Please provide at least one preferred time")
        return

    preferred_times_str = "; ".join(preferred_times)

    # Step 5: Save to database
    created_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    cursor.execute("""
        INSERT INTO EngagementRequests
        (parent_id, teacher_id, student_id, request_type, subject, message, preferred_times, status, created_at)
        VALUES (?, ?, ?, 'meeting', ?, ?, ?, 'pending', ?)
    """, (parent_id, teacher_id, student_id, subject, message, preferred_times_str, created_at))
    conn.commit()

    request_id = cursor.lastrowid

    # Get teacher email
    cursor.execute("""
        SELECT u.email FROM Teachers t
        JOIN Users u ON t.user_id = u.user_id
        WHERE t.teacher_id = ?
    """, (teacher_id,))
    teacher_email = cursor.fetchone()[0]

    # Confirmation
    print("\n" + "=" * 60)
    print("✅ MEETING REQUEST SUBMITTED!")
    print("=" * 60)
    print(f"   Request ID: {request_id}")
    print(f"   Teacher: {teacher_name}")
    print(f"   Student: {student_name}")
    print(f"   Status: Pending Teacher Approval")
    print(f"\n📋 Meeting Details:")
    print(f"   Subject: {subject}")
    if message:
        print(f"   Additional Notes: {message}")
    print(f"   Preferred Times:")
    for time in preferred_times:
        print(f"      • {time}")
    print(f"\n📧 Notification sent to: {teacher_email}")
    print(f"\n💡 The teacher will review and respond with a confirmed time.")


def parent_view_requests():
    """
    TEXT-BASED: View all engagement requests and their status
    """
    # Check if logged in as parent
    if not current_session or current_session['role'] != 'parent':
        print("❌ This feature is only available to parents. Please login as a parent first.")
        return

    parent_id = current_session['parent_id']

    print("\n" + "=" * 60)
    print("📋 MY ENGAGEMENT REQUESTS")
    print("=" * 60)

    query = """
        SELECT
            er.request_id,
            er.request_type,
            ut.name as teacher_name,
            us.name as student_name,
            er.subject,
            er.status,
            er.created_at,
            er.teacher_response,
            er.preferred_times,
            er.message
        FROM EngagementRequests er
        JOIN Teachers t ON er.teacher_id = t.teacher_id
        JOIN Users ut ON t.user_id = ut.user_id
        JOIN Students s ON er.student_id = s.student_id
        JOIN Users us ON s.user_id = us.user_id
        WHERE er.parent_id = ?
        ORDER BY er.created_at DESC
    """

    requests_df = pd.read_sql_query(query, conn, params=(parent_id,))

    if len(requests_df) == 0:
        print("\n📭 You have no engagement requests yet.")
        print("💡 Use parent_contact_teacher() or parent_request_meeting() to get started!")
        return

    print(f"\n📊 Summary:")
    print(f"   Total Requests: {len(requests_df)}")
    print(f"   ⏳ Pending: {len(requests_df[requests_df['status'] == 'pending'])}")
    print(f"   ✅ Approved: {len(requests_df[requests_df['status'] == 'approved'])}")
    print(f"   ✔️  Completed: {len(requests_df[requests_df['status'] == 'completed'])}")
    print(f"   ❌ Declined: {len(requests_df[requests_df['status'] == 'declined'])}")

    # Display each request with details
    for _, req in requests_df.iterrows():
        print("\n" + "-" * 60)
        print(f"🆔 Request #{req['request_id']} - {req['request_type'].upper()}")
        print(f"   Teacher: {req['teacher_name']}")
        print(f"   Student: {req['student_name']}")
        print(f"   Subject: {req['subject']}")
        print(f"   Status: {req['status'].upper()}")
        print(f"   Submitted: {req['created_at']}")

        if req['message']:
            print(f"   Your Message: {req['message']}")

        if req['request_type'] == 'meeting' and req['preferred_times']:
            print(f"   Preferred Times: {req['preferred_times']}")

        if req['teacher_response']:
            print(f"   📬 Teacher Response: {req['teacher_response']}")
        elif req['status'] == 'pending':
            print(f"   ⏳ Waiting for teacher response...")

    print("\n" + "=" * 60)




```
# This is formatted as code
```

# Section 9: Utility Functions

In [ ]:
def get_user_info():
    """Display current logged-in user info"""
    if current_session:
        print(f"Logged in as: {current_session['name']} ({current_session['role']})")
    else:
        print("Not logged in")

def view_all_test_accounts():
    """Helper function to view all test account credentials"""
    print("=" * 60)
    print("🔑 TEST ACCOUNT CREDENTIALS")
    print("=" * 60)
    print("\nAll accounts use password: password123\n")

    # Students
    print("👨‍🎓 STUDENTS:")
    query = "SELECT email, name FROM Users WHERE role='student' LIMIT 5"
    df = pd.read_sql_query(query, conn)
    for _, row in df.iterrows():
        print(f"   {row['email']} - {row['name']}")

    # Parents
    print("\n👨‍👩‍👧 PARENTS:")
    query = "SELECT email, name FROM Users WHERE role='parent' LIMIT 5"
    df = pd.read_sql_query(query, conn)
    for _, row in df.iterrows():
        print(f"   {row['email']} - {row['name']}")

    # Teachers
    print("\n👨‍🏫 TEACHERS:")
    query = "SELECT email, name FROM Users WHERE role='teacher'"
    df = pd.read_sql_query(query, conn)
    for _, row in df.iterrows():
        print(f"   {row['email']} - {row['name']}")

    # Admins
    print("\n👔 ADMINS:")
    query = "SELECT email, name FROM Users WHERE role='admin'"
    df = pd.read_sql_query(query, conn)
    for _, row in df.iterrows():
        print(f"   {row['email']} - {row['name']}")

In [ ]:
# ========================
# 🏫 AFTER HOURS SYSTEM
# ========================

import os
import sqlite3
from datetime import datetime, timedelta
import pytz
import pandas as pd
import uuid
import smtplib
from email.message import EmailMessage
from typing import Optional, Tuple
from IPython.display import display, clear_output
import ipywidgets as widgets

# ------------------ DATABASE SETUP -----------------------------
DB_PATH = "after_hours.db"

CREATE_SQL = """
PRAGMA foreign_keys = ON;

CREATE TABLE IF NOT EXISTS teachers (
    teacher_id TEXT PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT,
    timezone TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS availability (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    teacher_id TEXT NOT NULL,
    weekday INTEGER NOT NULL,
    start_hm TEXT NOT NULL,
    end_hm TEXT NOT NULL,
    FOREIGN KEY(teacher_id) REFERENCES teachers(teacher_id) ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS tickets (
    ticket_id TEXT PRIMARY KEY,
    teacher_id TEXT NOT NULL,
    submitter_name TEXT,
    submitter_email TEXT,
    submitter_id TEXT,
    question TEXT,
    submitted_at_utc TEXT,
    status TEXT,
    scheduled_slot_utc TEXT,
    ticket_notes TEXT,
    FOREIGN KEY(teacher_id) REFERENCES teachers(teacher_id) ON DELETE CASCADE
);
"""

# ------------------ UTILITY HELPERS ----------------------------
def _now_utc():
    return datetime.utcnow().replace(tzinfo=pytz.UTC)

def _parse_hm(hm: str):
    hh, mm = hm.split(":")
    return int(hh), int(mm)

def _localize(naive_dt: datetime, tzname: str):
    tz = pytz.timezone(tzname)
    if naive_dt.tzinfo is None:
        return tz.localize(naive_dt)
    return naive_dt.astimezone(tz)

# ------------------ CORE SYSTEM -------------------------------
class AfterHoursSystem:
    def __init__(self, db_path: str = DB_PATH):
        self.db_path = db_path
        self._init_db()

    def _init_db(self):
        conn = sqlite3.connect(self.db_path)
        cur = conn.cursor()
        cur.executescript(CREATE_SQL)
        conn.commit()
        conn.close()

    # --- Teacher setup ---
    def add_teacher(self, name: str, timezone: str, email: Optional[str] = None) -> str:
        tid = str(uuid.uuid4())
        conn = sqlite3.connect(self.db_path)
        cur = conn.cursor()
        cur.execute("INSERT INTO teachers(teacher_id,name,email,timezone) VALUES (?,?,?,?)",
                    (tid, name, email, timezone))
        conn.commit()
        conn.close()
        return tid

    def set_availability(self, teacher_id: str, weekday: int, start_hm: str, end_hm: str):
        conn = sqlite3.connect(self.db_path)
        cur = conn.cursor()
        cur.execute(
            "INSERT INTO availability(teacher_id,weekday,start_hm,end_hm) VALUES (?,?,?,?)",
            (teacher_id, weekday, start_hm, end_hm))
        conn.commit()
        conn.close()

    def get_teacher(self, teacher_id: str):
        conn = sqlite3.connect(self.db_path)
        cur = conn.cursor()
        cur.execute("SELECT teacher_id,name,email,timezone FROM teachers WHERE teacher_id=?",
                    (teacher_id,))
        row = cur.fetchone()
        conn.close()
        if not row:
            return None
        return {"teacher_id": row[0], "name": row[1], "email": row[2], "timezone": row[3]}

    def _get_availability_df(self, teacher_id: str) -> pd.DataFrame:
        conn = sqlite3.connect(self.db_path)
        df = pd.read_sql_query(
            "SELECT weekday, start_hm, end_hm FROM availability WHERE teacher_id=?",
            conn, params=(teacher_id,))
        conn.close()
        if df.empty:
            return df
        df['weekday'] = df['weekday'].astype(int)
        return df

    # --- Ticket handling ---
    def submit_ticket(self, teacher_id: str, submitter_name: str,
                      submitter_email: str, submitter_id: str,
                      question: str, submit_time: Optional[datetime] = None):
        if submit_time is None:
            submit_time = _now_utc()
        elif submit_time.tzinfo is None:
            submit_time = submit_time.replace(tzinfo=pytz.UTC)
        else:
            submit_time = submit_time.astimezone(pytz.UTC)

        teacher = self.get_teacher(teacher_id)
        if not teacher:
            raise ValueError("Teacher not found")

        ticket_id = str(uuid.uuid4())
        submitted_at_utc = submit_time.isoformat()

        proposed_slot_local = self.find_next_available_slot(teacher_id, submit_time)
        scheduled_slot_utc = None
        status = "QUEUED"
        if proposed_slot_local is not None:
            scheduled_slot_utc = proposed_slot_local.astimezone(pytz.UTC).isoformat()
            status = "SCHEDULED"

        conn = sqlite3.connect(self.db_path)
        cur = conn.cursor()
        cur.execute("""
            INSERT INTO tickets(ticket_id,teacher_id,submitter_name,submitter_email,submitter_id,
            question,submitted_at_utc,status,scheduled_slot_utc)
            VALUES (?,?,?,?,?,?,?,?,?)
        """, (ticket_id, teacher_id, submitter_name, submitter_email, submitter_id,
              question, submitted_at_utc, status, scheduled_slot_utc))
        conn.commit()
        conn.close()

        return {
            "ticket_id": ticket_id,
            "status": status,
            "teacher": teacher["name"],
            "scheduled_local": proposed_slot_local.isoformat() if proposed_slot_local else None
        }

    def find_next_available_slot(self, teacher_id: str, from_time_utc: Optional[datetime] = None,
                                 search_days: int = 14) -> Optional[datetime]:
        teacher = self.get_teacher(teacher_id)
        if not teacher:
            raise ValueError("Teacher not found")
        tzname = teacher["timezone"]
        tz = pytz.timezone(tzname)

        if from_time_utc is None:
            from_time_utc = _now_utc()
        elif from_time_utc.tzinfo is None:
            from_time_utc = from_time_utc.replace(tzinfo=pytz.UTC)
        else:
            from_time_utc = from_time_utc.astimezone(pytz.UTC)

        avail_df = self._get_availability_df(teacher_id)
        if avail_df.empty:
            return None

        conn = sqlite3.connect(self.db_path)
        tickets_df = pd.read_sql_query(
            "SELECT scheduled_slot_utc FROM tickets WHERE teacher_id=? AND scheduled_slot_utc IS NOT NULL",
            conn, params=(teacher_id,))
        conn.close()
        scheduled_local_starts = set()
        for _, row in tickets_df.iterrows():
            try:
                utc_dt = datetime.fromisoformat(row['scheduled_slot_utc']).astimezone(pytz.UTC)
                local = utc_dt.astimezone(tz)
                scheduled_local_starts.add(local.replace(second=0, microsecond=0))
            except Exception:
                continue

        start_utc = from_time_utc
        for day_offset in range(0, search_days + 1):
            day_candidate_utc = start_utc + timedelta(days=day_offset)
            local_candidate = day_candidate_utc.astimezone(tz)
            weekday = local_candidate.weekday()
            day_windows = avail_df[avail_df['weekday'] == weekday]
            if day_windows.empty:
                continue
            for _, win in day_windows.iterrows():
                sh, sm = _parse_hm(win['start_hm'])
                local_start_dt = tz.localize(datetime(year=local_candidate.year,
                                                      month=local_candidate.month,
                                                      day=local_candidate.day,
                                                      hour=sh, minute=sm))
                now_local = start_utc.astimezone(tz)
                if local_start_dt < now_local:
                    continue
                if local_start_dt.replace(second=0, microsecond=0) in scheduled_local_starts:
                    continue
                return local_start_dt
        return None

    def list_teachers(self):
        conn = sqlite3.connect(self.db_path)
        df = pd.read_sql_query("SELECT teacher_id, name FROM teachers", conn)
        conn.close()
        return df

# ------------------ INITIALIZE SYSTEM --------------------------
if os.path.exists(DB_PATH):
    os.remove(DB_PATH)
    print("🧹 Old database removed — starting fresh.")

sys = AfterHoursSystem()

teacher_names = ["Ms. Parker", "Mr. Lee", "Dr. Smith", "Ms. Johnson", "Mr. Patel"]
for t in teacher_names:
    tid = sys.add_teacher(t, "America/New_York", f"{t.lower().replace(' ', '')}@school.edu")
    for wd in range(5):  # Mon–Fri 9–5
        sys.set_availability(tid, wd, "09:00", "17:00")

print("✅ 5 default teachers created successfully.\n")

# ------------------ STUDENT / PARENT WIDGET --------------------
teacher_df = sys.list_teachers()
teacher_dropdown = widgets.Dropdown(
    options=[(row["name"], row["teacher_id"]) for _, row in teacher_df.iterrows()],
    description="Teacher:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="400px")
)

submitter_name = widgets.Text(description="Your Name:", layout=widgets.Layout(width="400px"))
submitter_email = widgets.Text(description="Your Email:", layout=widgets.Layout(width="400px"))
submitter_id = widgets.Text(description="Student/Parent ID:", layout=widgets.Layout(width="400px"))
question_box = widgets.Textarea(description="Question:", layout=widgets.Layout(width="400px", height="100px"))
submit_button = widgets.Button(description="Submit Question", button_style='success')
output = widgets.Output()

def on_submit_clicked(b):
    with output:
        clear_output()
        try:
            result = sys.submit_ticket(
                teacher_id=teacher_dropdown.value,
                submitter_name=submitter_name.value,
                submitter_email=submitter_email.value,
                submitter_id=submitter_id.value,
                question=question_box.value
            )
            print(f"🎫 Ticket submitted successfully!")
            print(f"Ticket ID: {result['ticket_id']}")
            if result['scheduled_local']:
                print(f"🕒 Scheduled Meet Time: {result['scheduled_local']}")
            else:
                print("No available slot found within the next 14 days.")
            print(f"Assigned Teacher: {result['teacher']}")
        except Exception as e:
            print("❌ Error submitting ticket:", str(e))

submit_button.on_click(on_submit_clicked)

display(widgets.VBox([
    widgets.HTML("<h3>📩 Submit After-Hours Question</h3>"),
    teacher_dropdown,
    submitter_name,
    submitter_email,
    submitter_id,
    question_box,
    submit_button,
    output
]))


🧹 Old database removed — starting fresh.
✅ 5 default teachers created successfully.



# Section 10: Main Execution

In [ ]:
print("\n" + "=" * 60)
print("✅ SYSTEM READY!")
print("=" * 60)
print("\n📚 How to use the system:")
print("\n1️⃣  LOGIN:")
print("   Run: login()")
print("   Use test credentials (all passwords: password123)")
print("\n2️⃣  FOR PARENTS - After logging in, use these functions:")
print("   parent_contact_teacher()  - Send a message to a teacher")
print("   parent_request_meeting()  - Request a meeting with a teacher")
print("   parent_view_requests()    - View all your request statuses")
print("\n3️⃣  OTHER FUNCTIONS:")
print("   logout()                  - Log out of current account")
print("   view_all_test_accounts()  - See all available test accounts")
print("   get_user_info()           - Check who is currently logged in")
print("\n💡 Example workflow:")
print("   1. Run: login()")
print("   2. Enter: parent1@email.com")
print("   3. Enter: password123")
print("   4. Run: parent_contact_teacher()")
print("   5. Follow the prompts!")
print("\n" + "=" * 60)


✅ SYSTEM READY!

📚 How to use the system:

1️⃣  LOGIN:
   Run: login()
   Use test credentials (all passwords: password123)

2️⃣  FOR PARENTS - After logging in, use these functions:
   parent_contact_teacher()  - Send a message to a teacher
   parent_request_meeting()  - Request a meeting with a teacher
   parent_view_requests()    - View all your request statuses

3️⃣  OTHER FUNCTIONS:
   logout()                  - Log out of current account
   view_all_test_accounts()  - See all available test accounts
   get_user_info()           - Check who is currently logged in

💡 Example workflow:
   1. Run: login()
   2. Enter: parent1@email.com
   3. Enter: password123
   4. Run: parent_contact_teacher()
   5. Follow the prompts!



In [ ]:



# to start the program just use the following function
login()

🍎 TEACHER DASHBOARD – Jeffrey Chavez


,student_name,course_name,assignment_name,grade_value,max_points,grade_date,category
0,Michele Williams,Algebra II,Midterm 5,7.9,10.0,2025-10-22,Quizzes
1,Holly Wood,Geometry,Final 10,11.8,20.0,2025-10-22,Homework
2,Cristian Santos,Geometry,Quiz 7,76.1,100.0,2025-10-21,Homework
3,Michele Williams,Algebra II,Quiz 6,74.8,100.0,2025-10-20,Homework
4,Dylan Miller,Algebra II,Test 5,9.7,10.0,2025-10-20,Quizzes
5,Allison Hill,Geometry,Project 6,90.6,100.0,2025-10-19,Quizzes
6,Abigail Shaffer,Algebra II,Project 2,17.7,20.0,2025-10-19,Quizzes
7,Carla Gray,Geometry,Midterm 3,89.1,100.0,2025-10-19,Quizzes
8,Allison Hill,Geometry,Homework 4,8.1,10.0,2025-10-18,Homework
9,Allison Hill,Geometry,Test 8,15.5,20.0,2025-10-17,Projects


In [ ]:
parent_contact_teacher()

❌ This feature is only available to parents. Please login as a parent first.


In [ ]:
logout()

✅ Logged out successfully!


In [ ]:
login()

🍎 TEACHER DASHBOARD – Jeffrey Chavez


,student_name,course_name,assignment_name,grade_value,max_points,grade_date,category
0,Michele Williams,Algebra II,Midterm 5,7.9,10.0,2025-10-22,Quizzes
1,Holly Wood,Geometry,Final 10,11.8,20.0,2025-10-22,Homework
2,Cristian Santos,Geometry,Quiz 7,76.1,100.0,2025-10-21,Homework
3,Michele Williams,Algebra II,Quiz 6,74.8,100.0,2025-10-20,Homework
4,Dylan Miller,Algebra II,Test 5,9.7,10.0,2025-10-20,Quizzes
5,Allison Hill,Geometry,Project 6,90.6,100.0,2025-10-19,Quizzes
6,Abigail Shaffer,Algebra II,Project 2,17.7,20.0,2025-10-19,Quizzes
7,Carla Gray,Geometry,Midterm 3,89.1,100.0,2025-10-19,Quizzes
8,Allison Hill,Geometry,Homework 4,8.1,10.0,2025-10-18,Homework
9,Allison Hill,Geometry,Test 8,15.5,20.0,2025-10-17,Projects


In [ ]:
# --- NEW: try RBAC login (same credentials as above) + shows grades
# Autumn Erwin
show_login_widget()

In [ ]:
# --- NEW: after successful RBAC login, route to the secure dashboard
# Autumn Erwin
show_my_dashboard()

🍎 TEACHER DASHBOARD – Jeffrey Chavez


,student_name,course_name,assignment_name,grade_value,max_points,grade_date,category
0,Michele Williams,Algebra II,Midterm 5,7.9,10.0,2025-10-22,Quizzes
1,Holly Wood,Geometry,Final 10,11.8,20.0,2025-10-22,Homework
2,Cristian Santos,Geometry,Quiz 7,76.1,100.0,2025-10-21,Homework
3,Michele Williams,Algebra II,Quiz 6,74.8,100.0,2025-10-20,Homework
4,Dylan Miller,Algebra II,Test 5,9.7,10.0,2025-10-20,Quizzes
5,Allison Hill,Geometry,Project 6,90.6,100.0,2025-10-19,Quizzes
6,Abigail Shaffer,Algebra II,Project 2,17.7,20.0,2025-10-19,Quizzes
7,Carla Gray,Geometry,Midterm 3,89.1,100.0,2025-10-19,Quizzes
8,Allison Hill,Geometry,Homework 4,8.1,10.0,2025-10-18,Homework
9,Allison Hill,Geometry,Test 8,15.5,20.0,2025-10-17,Projects


In [ ]:
# --- NEW: after RBAC login, shows role-filtered grades table with first 15 rows
# Autumn Erwin
secure_view_grades(16)

🔒 Role: teacher • Showing 16 of 106 rows


,grade_id,student_id,course_id,assignment_name,grade_value,max_points,grade_date,category,course_name,course_code,student_name
0,293,14,1,Midterm 5,7.9,10.0,2025-10-22,Quizzes,Algebra II,MATH201,Michele Williams
1,366,17,6,Final 10,11.8,20.0,2025-10-22,Homework,Geometry,MATH101,Holly Wood
2,104,5,6,Quiz 7,76.1,100.0,2025-10-21,Homework,Geometry,MATH101,Cristian Santos
3,297,14,1,Quiz 6,74.8,100.0,2025-10-20,Homework,Algebra II,MATH201,Michele Williams
4,318,15,1,Test 5,9.7,10.0,2025-10-20,Quizzes,Algebra II,MATH201,Dylan Miller
5,19,1,6,Project 6,90.6,100.0,2025-10-19,Quizzes,Geometry,MATH101,Allison Hill
6,132,7,1,Project 2,17.7,20.0,2025-10-19,Quizzes,Algebra II,MATH201,Abigail Shaffer
7,439,20,6,Midterm 3,89.1,100.0,2025-10-19,Quizzes,Geometry,MATH101,Carla Gray
8,15,1,6,Homework 4,8.1,10.0,2025-10-18,Homework,Geometry,MATH101,Allison Hill
9,17,1,6,Test 8,15.5,20.0,2025-10-17,Projects,Geometry,MATH101,Allison Hill


In [ ]:
view_announcements("All")
#Meetika Kanumukula


--- 📢 Announcements ---
No announcements available for your role at this time.



In [ ]:
login()
announcements_menu()
#Meetika Kanumukula

🔐 SCHOOL MANAGEMENT SYSTEM - LOGIN

📝 Test Accounts (all use password: password123):
   Student: student1@school.edu
   Parent:  parent1@email.com
   Teacher: teacher1@school.edu
   Admin:   admin1@school.edu


